# OpenDRIVE - Lanelet2 conversion

### 1. Imports

In [3]:
import os
import shutil
from pathlib import Path
from lxml import etree

# import sys
# sys.path.append(os.path.abspath(os.path.join(
#     os.path.dirname(__file__), 
#     '..',
# )))

from utils.conversion import prepConversionCRS, conductConversion
from utils.geometry import PointCoords, coords2XY, dist_2nodes, calAngleTriplePoints
from utils.postprocess import simplifyWayNodes, postprocessDownsamplingOSM

### 2. Declarations

In [4]:
# Input handling
input_dir = Path("./sample_data")
set_list = [
    "naive",
    "CARLA",
    "esmini"
]

# Output handling
output_dir = Path("./output/20250414")
os.makedirs(output_dir)

# Prep CRS conversion instance
scenario_location = prepConversionCRS()

# Downsampling params (will add to config later)
STRAIGHT_ANGLE_THRSH = 179.9            # Extremely strict angle threshold (trust me, 179 wasn't enough)
MIN_SEGMENT_DIST = 3.0                  # Minimum segment length accepted

### 3. Attempt conversion

In [ ]:
for set_name in set_list:

    print(f"\n=============== Working on {set_name} ===============\n")

    this_set_input_path = input_dir / set_name
    this_set_output_path = output_dir / set_name
    if not (os.path.exists(this_set_output_path)):
        os.makedirs(this_set_output_path)

    for input_file in os.listdir(this_set_input_path):

        # Input handling
        input_file_path = this_set_input_path / input_file
        print(f"\nConverting {input_file_path}")

        # Output handling
        input_file_tail_trimmed = ".".join(input_file.split(".")[ : -1])
        output_filename = f"converted_{input_file_tail_trimmed}.osm"
        output_path = this_set_output_path / output_filename
        
        # Conversion
        converted_osm = conductConversion(
            input_file = input_file_path, 
            scenario_location = scenario_location
        )

        # Conversion succeed!
        if (converted_osm):

            # Here comes my postprocessing
            downsamp_osm = postprocessDownsamplingOSM(
                converted_osm, 
                STRAIGHT_ANGLE_THRSH,
                MIN_SEGMENT_DIST
            )

            with open(output_path, "wb") as file_out:
                file_out.write(etree.tostring(
                    downsamp_osm, 
                    xml_declaration = True, 
                    encoding = "UTF-8", 
                    pretty_print = True
                ))
        
            print("Quest complete")